<a href="https://colab.research.google.com/github/ww-6/youtube-chatbot/blob/main/youtube-chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YouTube Chatbot

In [ ]:
!pip install -q -U langchain==0.1.0 langchain-community==0.0.9 sentence-transformers==2.3.0 \
gradio yt_dlp jq==1.6.0 chromadb==0.4.22

In [2]:
import torch

from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import JSONLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import AIMessage, HumanMessage
from langchain.memory.buffer_window import ConversationBufferWindowMemory
from langchain_community.llms import HuggingFaceHub
from langchain.chains import (
    LLMChain,
    StuffDocumentsChain,
    MapReduceDocumentsChain,
    ReduceDocumentsChain,
)

from gradio_client import Client
import gradio as gr
import yt_dlp
import json
import gc
import datetime
import os
import numpy as np

## 1. Prepare data

In [ ]:
whisper_jax_api = "https://sanchit-gandhi-whisper-jax.hf.space/"
whisper_jax = Client(whisper_jax_api)


def transcribe_audio(audio_path, task="transcribe", return_timestamps=True) -> str:
    text, runtime = whisper_jax.predict(
        audio_path,
        task,
        return_timestamps,
        api_name="/predict_1",
    )
    return text

In [4]:
def format_whisper_jax_output(
    whisper_jax_output: str, max_duration: int = 60
) -> list[dict]:
    """Whisper JAX outputs are in the format
    '[00:00.000 -> 00:00.000] text\n[00:00.000 -> 00:00.000] text'.

    Returns a list of dict with keys 'start', 'end', 'text'
    The segments from whisper jax output are merged to form paragraphs.

    `max_duration` controls how many seconds of the audio's transcripts are merged

    For example, if `max_duration`=60, in the final output, each segment is roughly
    60 seconds.
    """

    final_output = []
    max_duration = datetime.timedelta(seconds=max_duration)
    segments = whisper_jax_output.split("\n")
    current_start = datetime.datetime.strptime("00:00", "%M:%S")
    current_text = ""

    for i, seg in enumerate(segments):
        text = seg.split("]")[-1].strip()
        current_text += " " + text

        # Sometimes whisper jax returns None for timestamp
        try:
            end = datetime.datetime.strptime(seg[14:19], "%M:%S")
        except ValueError:
            end = current_start + max_duration

        if i == len(segments) - 1:
            final_output.append(
                {
                    "start": current_start.strftime("%H:%M:%S"),
                    "end": end.strftime("%H:%M:%S"),
                    "text": current_text.strip(),
                }
            )

        else:
            if end - current_start >= max_duration and current_text[-1] == ".":
                # If we have exceeded max duration, check whether we have
                # reached the end of a sentence. If not, keep merging.
                final_output.append(
                    {
                        "start": current_start.strftime("%H:%M:%S"),
                        "end": end.strftime("%H:%M:%S"),
                        "text": current_text.strip(),
                    }
                )

                # Update current start and text
                current_start = end
                current_text = ""

    return final_output

In [5]:
def yt_audio_to_text(url: str, max_duration: int = 60):
    """Given a YouTube url, download audio and transcribe it to text. Reformat
    the output from Whisper JAX and save the final result in a json file.
    """

    progress = gr.Progress()
    progress(0.1)

    with yt_dlp.YoutubeDL(
        {"extract_audio": True, "format": "bestaudio", "outtmpl": "audio.mp3"}
    ) as video:
        info_dict = video.extract_info(url, download=False)
        global video_title
        video_title = info_dict["title"]
        video.download(url)

    progress(0.4)
    audio_file = "audio.mp3"

    result = transcribe_audio(audio_file, return_timestamps=True)
    progress(0.7)

    result = format_whisper_jax_output(result, max_duration=max_duration)
    progress(0.9)

    with open("audio.json", "w") as f:
        json.dump(result, f)

    os.remove(audio_file)

## 2. Load data

In [6]:
def metadata_func(record: dict, metadata: dict) -> dict:
    """This function is used to tell the Langchain loader the keys that
    contain metadata and extract them.
    """
    metadata["start"] = record.get("start")
    metadata["end"] = record.get("end")
    metadata["source"] = metadata["start"] + " -> " + metadata["end"]

    return metadata


def load_data():
    loader = JSONLoader(
        file_path="audio.json",
        jq_schema=".[]",
        content_key="text",
        metadata_func=metadata_func,
    )

    data = loader.load()
    os.remove("audio.json")

    return data

## 3. Create embeddings and vector store

In [ ]:
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
device = "cuda" if torch.cuda.is_available() else "cpu"
embedding_model_kwargs = {"device": device}

embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name, model_kwargs=embedding_model_kwargs
)


def create_vectordb(data, n_retrieved_docs: int, collection_name="YouTube"):
    """Returns a retriever which is used to fetch relevant documents from
    the vector database.

    `n_retrieved_docs` is the number of retrieved documents.
    """

    vectordb = Chroma.from_documents(
        documents=data, embedding=embeddings, collection_name=collection_name
    )
    n_docs = len(vectordb.get()["ids"])
    retriever = vectordb.as_retriever(
        search_type="mmr", search_kwargs={"k": n_retrieved_docs, "fetch_k": n_docs}
    )

    return retriever

## 4. Load LLM

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""
repo_id = "mistralai/Mistral-7B-Instruct-v0.3"
llm = HuggingFaceHub(repo_id=repo_id, model_kwargs={"max_new_tokens": 1000})
llm.client.api_url = 'https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3'

## 5. Summarisation
We will use the map-reduce method for summarisation. The documents will be first summarised individually (map step). Then their summaries are combined and reduced further to give a single global summary (reduce step).

In [9]:
# Map
map_template = """Summarise the following text:
{docs}

Answer:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

In [10]:
# Reduce
reduce_template = """The following is a set of summaries:
{docs}

Take these and distill it into a final, consolidated summary of the main themes \
in 150 words or less.

Answer:"""

reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to llm
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

After summarising individual documents, the combined summaries could still exceed the max tokens. In that case, we pass in the summaries in batches and create batched summaries. Once all the batched summaries combined are less than the max tokens, we pass them all to the LLM.

In [11]:
# Combines and iteravely reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

Finally, we combine our map and reduce chains into one.

In [12]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)


def get_summary(documents) -> str:
    summary = map_reduce_chain.invoke(documents, return_only_outputs=True)
    return summary["output_text"].strip()

## 6. Q&A

### 6.1 Contextualising the question
The latest user question may make reference to information in the chat history but we don't want to use the entire history to search for answer in the database because not all of the information is relevant. We want to reformulate the question such that it contains only the relevant information, and can be understood without the chat history.

In [13]:
contextualise_q_prompt = PromptTemplate.from_template(
    """Given a chat history and the latest user question \
    which might reference the chat history, formulate a \
    standalone question that can be understood without \
    the chat history. Do NOT answer the question, just \
    reformulate it if needed and otherwise return it as is.

    Chat history: {chat_history}

    Question: {question}

    Answer:
    """
)

contextualise_q_chain = contextualise_q_prompt | llm

Let's test our `contextualise_q_chain` and see if it does what we want. Suppose we have the following conversation:
    
> Human: What is the capital city of Australia?

> AI: Canberra.

> Human: How far is it from Sydney?

We know that "it" here refers to Canberra. Let's see if the LLM can rephrase the question to include Canberra in the question.

In [14]:
# Test
chat_history = []
first_question = "What is the capital city of Australia?"
ai_msg = "Canberra"
chat_history.extend([HumanMessage(content=first_question), AIMessage(content=ai_msg)])

second_question = "How far is it from Sydney?"
answer = contextualise_q_chain.invoke(
    {"question": second_question, "chat_history": chat_history}
)
print(answer)


    What is the distance between Sydney and Canberra?


### 6.2 Standalone question chain
Reformulating the question takes time and not all questions need contextualising. To speed up the process, we add a sub-chain which determines whether a question needs contextualising or not. If the question is a standalone question, then we can use the user input directly without modifications.

In [15]:
standalone_prompt = PromptTemplate.from_template(
    """Given a chat history and the latest user question, \
    identify whether the question is a standalone question \
    or the question references the chat history. Answer 'yes' \
    if the question is a standalone question, and 'no' if the \
    question references the chat history. Do not answer \
    anything other than 'yes' or 'no'.

    Chat history:
    {chat_history}

    Question:
    {question}

    Answer:
    """
)


def format_output(answer: str) -> str:
    """All lower case and remove all whitespace to ensure
    that the answer given by the LLM is either 'yes' or 'no'.
    """
    return "".join(answer.lower().split())


standalone_chain = standalone_prompt | llm | format_output

Let's test the `standalone_chain` using the same example as before. Recall that the question "How far is it from Sydney?" refers to the answer given by the AI so it is not a standalone question. Let's see if the LLM can pick up on that.

In [16]:
# Test
answer = standalone_chain.invoke(
    {"question": second_question, "chat_history": chat_history}
)
print(answer)

no


### 6.3 Q&A chain
Finally, we can build our Q&A chain. The process goes as follows:


1.   Check whether the latest user question needs contextualising or not, using the `standalone_chain`.
2.   If the question is a standalone question, use it to retrieve documents from the database. Otherwise, reformulate the question using `contextualise_q_chain` to get a contextualised question and use it to retrieve documents from the database.
3.   Pass the retrieved documents as `context`, together with the contextualised question to the LLM to receive an answer.


We do not want the LLM to use outside knowledge so we tell the LLM that it can only use the information given in `context` to answer the question.



In [17]:
qa_prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. \
    ONLY use the following context to answer the question. \
    Do NOT answer with information that is not contained in \
    the context. If you don't know the answer, just say:\
    "Sorry, I cannot find the answer to that question in the video."

    Context:
    {context}

    Question:
    {question}

    Answer:
    """
)

In [18]:
class YouTubeChatbot:
    instance_count = 0

    def __init__(
        self,
        n_sources: int = 3,
        n_retrieved_docs: int = 5,
        timestamp_interval: datetime.timedelta = datetime.timedelta(minutes=2),
        memory: int = 5,
    ):
        YouTubeChatbot.instance_count += 1
        self.chatbot_id = YouTubeChatbot.instance_count
        self.n_sources = n_sources
        self.n_retrieved_docs = n_retrieved_docs
        self.timestamp_interval = timestamp_interval
        self.chat_history = ConversationBufferWindowMemory(k=memory)
        self.retriever = None
        self.qa_chain = None


    def format_docs(self, docs: list) -> str:
        """Combine documents into a single string which will be included
        in the prompt given to the LLM.
        """
        self.sources = [doc.metadata["start"] for doc in docs]

        return "\n\n".join(doc.page_content for doc in docs)


    def standalone_question(self, input_: dict) -> str:
        """If the question is a not a standalone question,
        run contextualise_q_chain.
        """
        if input_["standalone"] == "yes":
            return contextualise_q_chain
        else:
            return input_["question"]


    def format_answer(self, answer: str) -> str:
        """Add timestamps to answers.
        """
        if "cannot find the answer" in answer:
            return answer.strip()
        else:
            timestamps = self.filter_timestamps()
            answer_with_sources = (
                answer.strip() + " You can find more information "
                "at these timestamps: {}.".format(", ".join(timestamps))
            )
            return answer_with_sources


    def filter_timestamps(self) -> list[str]:
        """Returns a list of timestamps with length less or
        equal to `n_sources`. The timestamps are at least an
        `timestamp_interval` apart. This prevents returning
        a list of timestamps that are too close together.
        """

        filtered_timestamps = np.array(
            [datetime.datetime.strptime(self.sources[0], "%H:%M:%S")]
        )

        i = 1

        while len(filtered_timestamps) < self.n_sources:
            try:
                new_timestamp = datetime.datetime.strptime(self.sources[i], "%H:%M:%S")
            except IndexError:
                break

            absolute_time_difference = abs(new_timestamp - filtered_timestamps)

            if all(absolute_time_difference >= self.timestamp_interval):
                filtered_timestamps = np.append(filtered_timestamps, new_timestamp)

            i += 1

        filtered_timestamps = [
            timestamp.strftime("%H:%M:%S") for timestamp in filtered_timestamps
        ]
        filtered_timestamps.sort()

        return filtered_timestamps


    def process_video(self, url: str, data=None, retriever=None):
        """Given a YouTube URL, transcribe YouTube audio to text.
        Then set up the vector database.
        """
        yt_audio_to_text(url)
        data = load_data()

        if retriever is not None:
            # If we already have documents in the vector store, delete them.
            ids = retriever.vectorstore.get()["ids"]
            retriever.vectorstore.delete(ids)

        retriever = create_vectordb(
            data, self.n_retrieved_docs,
            collection_name=f"Chatbot{self.chatbot_id}"
        )

        return url, data, retriever


    def setup_qa_chain(self, retriever, qa_chain=None):
        qa_chain = (
            RunnablePassthrough.assign(standalone=standalone_chain)
            | {
                "question": self.standalone_question,
                "context": self.standalone_question | retriever | self.format_docs,
            }
            | qa_prompt
            | llm
        )

        return retriever, qa_chain


    def setup_chatbot(self, url: str):
        _, _, self.retriever = self.process_video(url=url, retriever=self.retriever)
        _, self.qa_chain = self.setup_qa_chain(retriever=self.retriever)


    def get_answer(self, question: str) -> str:
        try:
            ai_msg = self.qa_chain.invoke(
                {"question": question, "chat_history": self.chat_history}
            )
        except AttributeError:
            raise AttributeError(
                "You haven't setup the chatbot yet. "
                "Setup the chatbot by calling the "
                "instance method `setup_chatbot`."
            )

        self.chat_history.save_context({"question": question}, {"answer": ai_msg})

        answer = self.format_answer(ai_msg)

        return answer

### 6.4 Test our chatbot

In [ ]:
chatbot = YouTubeChatbot(
    n_sources=3,
    n_retrieved_docs=5,
    timestamp_interval=datetime.timedelta(minutes=2),
    memory=5,
)

url = "https://www.youtube.com/watch?v=SZorAJ4I-sA"
chatbot.setup_chatbot(url)

#### 6.4.1 Test 1: A question that cannot be answered by the video alone.

In [20]:
question = "What is the capital city of Australia?"
answer = chatbot.get_answer(question)
print(answer)

Sorry, I cannot find the answer to that question in the video.


#### 6.4.2 Test 2: A question that can be answered by the video.

In [21]:
question = "In what ways did transformers improve upon RNN?"
answer = chatbot.get_answer(question)
print(answer)

Transformers improved upon RNN in several ways. First, they were able to efficiently parallelize, which meant that with the right hardware, they could train some really big models. This allowed them to handle large sequences of text, like long paragraphs or essays, without forgetting what happened in the beginning. Second, transformers were easier to train than RNNs because they processed words sequentially, which meant that they could paralyze well and speed up training. Finally, transformers were able to build up an internal representation or understanding of language automatically as they analyzed tons of text data, which made them useful for a variety of language tasks. You can find more information at these timestamps: 00:00:00, 00:02:07, 00:05:12.


In [ ]:
del chatbot
gc.collect()

## 7. Web app

In [23]:
class YouTubeChatbotApp(YouTubeChatbot):
    def __init__(
        self,
        n_sources: int,
        n_retrieved_docs: int,
        timestamp_interval: datetime.timedelta,
        memory: int,
        default_youtube_url: str,
    ):
        super().__init__(n_sources, n_retrieved_docs, timestamp_interval, memory)

        self.default_youtube_url = default_youtube_url
        self.memory = memory
        self.chat_history = None
        self.data = None
        self.retriever = None
        self.qa_chain = None

        # Gradio components
        self.url_input = None
        self.url_button = None
        self.app_chat_history = None
        self.chatbot = None
        self.user_input = None
        self.clear_button = None

    def greet(self, data, app_chat_history) -> dict:
        """Summarise the video and greet the user.
        """
        summary_message = f'Here is a summary of the video "{video_title}":'
        app_chat_history.append((None, summary_message))

        summary = get_summary(data)
        self.data = gr.State(None)
        app_chat_history.append((None, summary))

        greeting_message = (
            "You can ask me anything about the video. " "I will do my best to answer!"
        )
        app_chat_history.append((None, greeting_message))

        return {self.app_chat_history: app_chat_history, self.chatbot: app_chat_history}

    def question(self, user_question: str, app_chat_history) -> dict:
        """Display the question asked by the user in the chat window,
        and delete from the input textbox.
        """
        app_chat_history.append((user_question, None))
        return {
            self.user_input: "",
            self.app_chat_history: app_chat_history,
            self.chatbot: app_chat_history,
        }

    def respond(self, qa_chain, chat_history, app_chat_history) -> dict:
        """Respond to user's latest question"""
        question = app_chat_history[-1][0]

        try:
            ai_msg = qa_chain.invoke(
                {"question": question, "chat_history": chat_history}
            )
        except AttributeError:
            raise gr.Error(
                "You need to process the video " "first by pressing the `Go` button."
            )

        chat_history.save_context({"question": question}, {"answer": ai_msg})

        answer = self.format_answer(ai_msg)

        app_chat_history.append((None, answer))

        return {
            self.qa_chain: qa_chain,
            self.chat_history: chat_history,
            self.app_chat_history: app_chat_history,
            self.chatbot: app_chat_history,
        }

    def clear_chat_history(self, chat_history, app_chat_history):
        chat_history.clear()
        app_chat_history = []
        return {
            self.chat_history: chat_history,
            self.app_chat_history: app_chat_history,
            self.chatbot: app_chat_history,
        }

    def launch(self, **kwargs):
        with gr.Blocks() as demo:
            self.chat_history = gr.State(ConversationBufferWindowMemory(k=self.memory))
            self.app_chat_history = gr.State([])
            self.data = gr.State()
            self.retriever = gr.State()
            self.qa_chain = gr.State()

            # App structure
            with gr.Row():
                self.url_input = gr.Textbox(
                    value=self.default_youtube_url, label="YouTube URL", scale=5
                )
                self.url_button = gr.Button(value="Go", scale=1)

            self.chatbot = gr.Chatbot()
            self.user_input = gr.Textbox(label="Ask a question:")
            self.clear_button = gr.Button(value="Clear")


            # App actions

            # When a new url is given, clear past chat history and process
            # the new video. Set up the Q&A chain with the new video's data.
            # Provide a summary of the new video.
            self.url_button.click(
                self.clear_chat_history,
                inputs=[self.chat_history, self.app_chat_history],
                outputs=[self.chat_history, self.app_chat_history, self.chatbot],
                trigger_mode="once",
            ).then(
                self.process_video,
                inputs=[self.url_input, self.data, self.retriever],
                outputs=[self.url_input, self.data, self.retriever],
            ).then(
                self.setup_qa_chain,
                inputs=[self.retriever, self.qa_chain],
                outputs=[self.retriever, self.qa_chain],
            ).then(
                self.greet,
                inputs=[self.data, self.app_chat_history],
                outputs=[self.app_chat_history, self.chatbot],
            )

            # When a user asks a question, display the question in the chat
            # window and remove it from the text input area. Then respond
            # with the Q&A chain.
            self.user_input.submit(
                self.question,
                inputs=[self.user_input, self.app_chat_history],
                outputs=[self.user_input, self.app_chat_history, self.chatbot],
                queue=False,
            ).then(
                self.respond,
                inputs=[self.qa_chain, self.chat_history, self.app_chat_history],
                outputs=[
                    self.qa_chain,
                    self.chat_history,
                    self.app_chat_history,
                    self.chatbot,
                ],
            )

            # When the `Clear` button is clicked, clear the chat history from
            # the chat window.
            self.clear_button.click(
                self.clear_chat_history,
                inputs=[self.chat_history, self.app_chat_history],
                outputs=[self.chat_history, self.app_chat_history, self.chatbot],
                queue=False,
            )

        demo.launch(**kwargs)

In [ ]:
app = YouTubeChatbotApp(
    n_sources=3,
    n_retrieved_docs=5,
    timestamp_interval=datetime.timedelta(minutes=2),
    memory=5,
    default_youtube_url="https://www.youtube.com/watch?v=SZorAJ4I-sA",
)

app.launch(debug=True)